**FSDS**: Virtual Environments and Installs v0.1.1

**Author**: Bernie Hogan

# Installing Geopandas 

There are an extensive number of tutorials online for a variety  of GIS tools, including geopandas. The common approach recommended seems to be to download from `conda` which is the anaconda package manager. However, on my computer I tend to get such at the 'solving the dependencies' stage. This looks something like this: 

~~~ bash
(base) work@Bernies-MacBook-Air Downloads % conda install geopandas

Collecting package metadata (current_repodata.json): done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
~~~

This is because conda is trying to determine if I have the right versions of packages to install. There are a variety of ways to address this specific issue, but I decided to sidestep it by installing the dependencies directly. These have to be installed in the following order: 
- GDAL: First on mac / *Nix, you must install ['homebrew'](https://brew.sh/). Then you can install GDAL from source using the following commands: 
  - `% brew install gdal --HEAD`
  - `% brew install gdal`
  Thereafter the rest of the packages are fairly straightforward.
- pyproj: `pip install pyproj`
- mapclassify: `pip install mapclassify`
- Fiona: `pip install Fiona`
- Shapely: `pip install Shapely`
- geopandas: `pip install geopandas`

There are many different versions of each of these programs and understandably this installation order could change over time. If this does not work, consider searching online for more recent tutorials. 

# Virtual environments and the trials of installation

Up until this point we have been assuming that there is one ideal version of Python, the one that comes with Anaconda and is installed on our machine. However, that is only going to get us so far. This is especially the case with relatively cutting edge software built by vast arrays of persons all working not in tandem. A common scenario might happen when you want to install new software whereby the dependencies of the software make it such that you simply can't run some packages without changing some other packages. This leads to a regress of installing on top of new installs on top of etc. 

Here's an example error from some truncated code where I simply typed `conda install geopandas`: 

~~~ bash 
(base) work@Personals-MacBook-Air ~ % conda install geoplot -c conda-forge
Collecting package metadata (current_repodata.json): done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Collecting package metadata (repodata.json): done
Solving environment: / failed with initial frozen solve. Retrying with flexible solve.

...

Package wheel conflicts for:
pip -> wheel
anaconda==2020.07 -> wheel==0.34.2[build='py38_0|py37_0|py36_0']
anaconda==2020.07 -> pip==20.1.1=py37_1 -> wheel
python=3.8 -> pip -> wheel
~~~

And it failed. To note, those ellipses actually signal a very large number of dependency clashes that happened. Why is it so complicated to install software and how can I avoid this? 

Below I will talk about two approaches to managing packages/modules, the first seems to work pretty reliably on mac. It works on Windows as well, but I had considerable trouble integrating it into Jupyter. This however gives me the opportunity to talk about how to build Jupyter from a fresh install with miniconda. So for the very brave who want to see geopandas on Windows can use this to rebuild their Jupyter environment (and even on mac, too if you want to see how to build a leaner more bespoke conda and jupyter). 

## Conda vs. Pip vs. Building from source 

Python is not merely a programming language but also a compiler for running code within the Python language. That means that it will expect code of a certain version and code from a later version will not run. We can check which version of Python we are running with `import sys; sys.version`

In [2]:
import sys
sys.version

'3.9.0 | packaged by conda-forge | (default, Oct 14 2020, 22:56:29) \n[Clang 10.0.1 ]'

Generally speaking when we want to install packages for Python we want to make sure that we are installing to the correct version. Depending on your operating system, there might be a number of versions of Python available. This is why when we install within Jupyter we will want to use the following code: 

~~~ python 

try: 
    import <package> 
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install <package>
    import <package>
~~~

It is because the `sys` module enables us to use commands that talk about the Python engine (which is the 'system' we are using to run Python commands). We could have just said `!pip install <package>` but if you have multiple versions of Python on your computer, you might end up installing it for the wrong one. What you usually want is the version that you're using here in this notebook (i.e. the version shown in the lower-left and upper-right corners). It probably says "Python 3 | Idle" right now. 

### `pip` - The official Python installer
Notice that above I used `pip` to install a package. `pip` stands for the Python Installer Program. It is maintained by the Python Software Foundation. Pip simplifies the act of installing. Prior to pip it was conventional to download an archive, unzip it, run `python setup.py` which will build a version of the package for your version of Python, and you hope for the best. Pip instead checks for existing versions of that program or library, checks the version of Python, gets the appropriate archive from the server where possible and installs it. What server you might ask? Generally, it is from PyPI which is where you can host Python libraries. PyPI is the Python equivalent of `cran` for R archives. 

### Pip and requirements 

Pip doesn't just install packages but operates as a package manager, too. Herein there are two commands worth noting: `freeze` and `list`. If you run `pip list` it will print a list of the installed packages for this version of Python. It is a superset of those that you can install with `pip`. On the other hand, if you use `pip freeze`, it will export a list printed in a handy format that you can use to rebuild this specific version of python. How? By including this output in a file called `requirements.txt`. So you can use this when rebuilding Python by using the following: 

In [4]:
# This works in Jupyter lab and creates the requirements.txt file in the pwd.
# To run this in the terminal, drop the ! 
!pip freeze > requirements.txt

And then in the new fresh install you can type: 

~~~ bash
pip install -r requirements.txt
~~~

### `conda` - The Anaconda Python installer
Typically, we are better off installing programs from `conda` where possible if we are using Jupyter Lab. Conda does an excellent job of hosting and supporting programs and it handles issues with installs a little more gracefully than `pip`. We will find that sometimes however, conda can be a little behind pip in terms of upgrades. So now that Python is on 3.9, while many packages only officially support Python 3.8.

You will note that many of the packages installed by conda might appear in the `pip list` but not `pip freeze` since freeze only includes those that `pip` can install. 

Conda brings in a version of a module that works nicely with Anaconda versions of Python. Be default it looks in the main branch of conda which has the versions you're likely to find in the standard stable Anaconda distribution. However, it is recommended when building from conda to consider whether you should download instead from Conda-Forge. Conda-forge includes many more modules/libraries. Generally if you start with one you should stick with it, so for the rest of this tutorial we will be using the `-c conda-forge` flag like so: 

~~~ bash 
% conda install <package-name> -c conda-forge
~~~

Or if we are doing it from within Jupyter lab: 

~~~ python
!conda install --yes --prefix {sys.prefix} <package-name> -c conda-forge
~~~

The `--yes` flag says just say yes to 'do you wish to install' and prefix helps to ensure you're installing for _this_ version of conda in case you have other virtual environments running (which we will cover below). 

### Compiling from source 
Every so often, you will want to install a library that isn't available from PyPI. In which case you will want to 'build it from source'. What is actually happening here is that it usually builds a program called a 'wheel'. These wheels are what Python loads. They are built versions of the Python code. When you are asking for a library using `pip` it will actually search PyPI for whether there is already a built wheel or whether you (meaning your computer) will have to build it yourself. An example of this is the Postgres library for Python. This library `psycopg2`, which is slightly more user friendly and faster than the generic SQL interface from `sqlalchemy` has pre-built binaries (wheels) for Windows, but Mac and Linux users would have to build it from scratch. Pip will handle this for us.  

If you have to build a package from source, normally you would download the source code to your computer, navigate to that director in a python-enabled terminal and run `python setup.py`. On windows you might need to open the terminal as administrator, buuuut you shouldn't need to type `sudo python setup.py` or you might have existing permissions issues with packages. 

# Using a Virtual Environment to solve dependency issues
> Windows Users: 
> Please note that there are two versions of the PowerShell now, the standard windows PowerShell which is in blue and the Anaconda PowerShell. Indeed, this is no longer a DOS cmd prompt but is much more like Unix (back/forward slashes notwithstanding). You will want to launch the Anaconda PowerShell for this to work properly.

If we have a version of software that doesn't work according to what we need, we might want to install a new version. However, what if that breaks our install path? We can use a 'virtual environment' in order to create multiple instances of Python and then use those instances for different purposes. It can get a bit messy on your computer admittedly, but it is a good way to manage the many complex dependencies that different programs present. 

I am bringing this up now because I have had some real significant issues installing `geopandas` and `geoplot` on my computer because my current Python is 3.8, but `geopandas` via `conda` wanted 3.9. However, to make matters more complicated, then when I did that some of the _other_ modules that I wanted did not want Python 3.9 but wanted an earlier version. So, I will walk through that as an issue as well. 

First let's create a couple virtual environments, then let's choose one for our `geopandas` / `geoplot` install. We will have to do this through the terminal, but I will address the instructions here. Please note, due to the nature of this particular task, it is pretty difficult to address all contingencies across systems. However, the exact same commands appear to work in the Windows Powershell and the Mac Terminal. 

## Installing Virtual Environments using `venv` 

You might already have venv installed as a part of the anaconda distribution of Python. It is not a library that you import into Python, but a way in order to create multiple instances of Python. In Python 2.* it was called `virtualenv`, but now it's simply called `venv`. Try it out by going to a terminal or a Powershell window and typing the following: 

~~~ bash 
python -m venv exampleenv
~~~ 

Which will create a new virtual environment called `exampleenv`. It is located in the directory where you ran the command above. It will be the same version of python that you used in order to create the virtual environment (in this case it is Python 3.8.3 which is my default version when I type `python` on the command line). 

This will not, however, allow you to directly create a virtual environment with a different version of Python. For that you'll need to still use `virtualenv` or `conda`. In using `virtualenv`, to have the desired version installed alongside your existing version, you can set a path to that version for the virtual environment like so: 
~~~ bash
virtualenv --python=/usr/bin/python2.6 <path/to/new/virtualenv/>
~~~ 

At this point perhaps you have already installed a variety of packages such as `seaborn` and `emoji`. If you type `python` on the terminal and then type `import seaborn` it should give you no errors. Now watch what happens when you 'activate' the virtual environment. 

Recall that we created the virtual environment as a series of files in the exampleenv directory. So to activate that version of Python, we would type: (assuming exampleenv is a folder in this directory) 
~~~ 
source exampleenv/bin/activate  
~~~ 
`source` is the command to say "this is the Python version we should use now". Notice then that preceding your command prompt should be `(exampleenv)` to signal that now you are in the virtual environment. Now whenever you use `pip` or `conda` it will install in that folder and be relevant for that environment only. In order to leave that environment and get back to your default you simply type `deactivate`. 

Have a look at this sample of commands from my computer: 
~~~ bash
(base) work@Personals-MacBook-Air ~ % source exampleenv/bin/activate
(exampleenv) (base) work@Personals-MacBook-Air ~ % python
Python 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ] :: Anaconda, Inc. on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import emoji
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'emoji'
>>> exit()
(exampleenv) (base) work@Personals-MacBook-Air ~ % deactivate
(base) work@Personals-MacBook-Air ~ % python
Python 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ] :: Anaconda, Inc. on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import emoji
>>>
>>> exit()
~~~


## Creating a virtual environment using `conda` 

If you already have the Anaconda distribution on your computer then you can further do this with `conda`. We are going to do this instead of venv here because it will allow us to use conda to update Python itself as well as do a nice install of geopandas. Then, at least on Mac/Unix (and in theory on Windows if you can sort out the activation process, which I struggled with), you can use this virtual environment as its own kernel for Jupyter Lab, which we will see afterwards. 

On the GeoPandas website it says that you really should install this software using `conda` rather than `pip`. So you might be tempted to simply type: 

~~~ 
conda install geopandas 
~~~ 

But having installed Anaconda as I have done, you'll find that there are already many packages in there with the wrong versions, thereby making it difficult to know how to upgrade. In fact, it might simply fail. Worse, it will take a long time to do so as it indexes all the dependencies, then reports them to you, and then fails giving you errors that look like the following: 

~~~ bash 
(base) work@MacBook-Air ~ %conda install -c conda-forge geopandas
Collecting package metadata (current_repodata.json): done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...
~~~ 

What has happened is that `frozen solve` meant that it "freezes" the packages (using `pip freeze` we saw above), sends those to Conda-Forge and says "hey, I have all these packages, what version of GeoPandas will work with this?" and it says "I can't find any version that will satisfy all the dependnencies...and if I start updating these packages other ones will break". 

So the way to sidestep all this is to create a fresh install, but not by getting rid of Python on your computer, but by using the `create -n <environment-name>` argument instead of `install`, and then installing the right packages with that instance. It will be slow. 

It is also considerably more powerful than `venv`, as you can see from [the many options](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html) it has including setting the version of Python. I use the following command, with a hat tip to this [GIS Stack Exchange post](https://gis.stackexchange.com/questions/358889/geopandas-installation-errors). 

~~~ bash
conda create -n geo-env -c conda-forge geopandas
~~~ 

By running this command it will create a new Python environment which we can refer to as `geo-env`. In fact, it should create the most recent Python environment (3.9) which might create some other issues that we can address shortly. But before we get to that, we need to be working in this environment, else anything we install won't be a part of this environment but of the default Python environment on our computer. 

To activate this new environment, which would be called `geo-env`, you would type: 

~~~ bash 
conda activate geo-env
~~~ 

And similar to above you should see `(geo-env)` prior to the command prompt. If so, you're in. At this point you can test that it worked by typing `python` to get the Python command prompt from within the virtual environment. Then type `import geopandas` which should now be installed in this instance of the Python environment. But we typically do not work in the Python command line but in either Python files or in Jupyter notebooks. So let's see how to use this new environment in Jupyter. 

## Using a different Virtual Kernel in Jupyter Lab

There is a python module called `ipykernel`. It comes with the full Anaconda install but we won't be able to access that from within a fresh virtual environment or a miniconda install. Thus we will need to install it ourselves. From within our virtual environment we would need to run: 

~~~ bash
pip install ipykernel
~~~ 

or # for fresh miniconda installs from scratch, try:

~~~
conda install jupyter -c conda-forge
~~~ 

If you install `jupyter` this way, ipykernel should come along for the ride. However, if this is inside a virtual environment on Mac / Unix there's no need to install Jupyter Lab. Why? Because we are just going embed this virtual environment as a kernel in our _existing_ Jupyter Lab, not try to create a parallel Jupyter Lab instance on our machine. 

Once `ipykernel` is installed in your environment, you can run it from within this environment with the flag `--user` to install this kernel in your standard Jupyter Lab instance. 

~~~ bash
python -m ipykernel install --user --name=env_name 
~~~

Ok now stop! If you are inside a virtual environment (for example you have `(geo-env)` at the beginning of your prompt), then you will need to deactivate this environment, since jupyter lab is not likely to be installed in the virtual environment. If you are in a virtual environment from venv it is simply `deactivate` in the prompt. If it is a conda virtual environment the command is `conda deactivate`. 

## Fresh install with Miniconda 

Above I mentioned doing a clean miniconda install. This is a useful albeit last ditch way to install `geopandas`. While it is useful to try and install `geopandas` within a virtual environment, sometimes particularly on windows the software in Jupyter simply can't see or run binaries that are installed in these virtual environments. You can tell this by getting the following errors/notifications when running a kernel:
- DLL not found errors (these will show up if you try to use a broken or unactivated kernel in Windows)
- From a base environment, when running the Python3.exe file inside a an envs directory, you'll see a warning that says "This Python interpreter is in a conda environment, but the environment has not been activated..." and provides a link to https://conda.io/activation.  

Then perhaps you'll want to do an entire fresh install of Python and build Jupyter Lab from the ground up. **Warning** - don't do this if you don't want to reinstall everything. If you are having some issues and want to start fresh, you should first use conda to download Anaconda-Clean: 

~~~ bash
conda install anaconda-clean
~~~

Then delete the anaconda folder (and in windows check add/remove programs for any versions of Python). At this point you should be Anaconda/Python free. Then download miniconda and run that installer. 

Should you do this, you'll have a fresh, lean version of conda that won't have Jupyter Lab installed. However, at this point you can run: 

~~~
conda install geopandas -c conda-forge
~~~ 
And it should build a version of Python that will work with geopandas. So once that's done, typing `python` in the terminal should allow you to then type `import geopandas` and receive no errors. The good news is that Pandas and numpy come along for free, but alas many others such as jupyterlab, beautifulsoup, and seaborn will have to be installed separately. 

You can use a prior mentioned `requirements.txt` alongside  file in order to bring in the full anaconda install, but I might suggest trying to install on a need to download basis using the syntax from above.

# Using a Different Kernel 

Jupyter Lab kernels do not need to be Python, virtual or otherwise. [This GitHub wiki page](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels) in the Jupyter repo shows a pretty regularly updated list of JupyterLab Kernels, of which there are over a hundred now. 

The most popular kernel is likely the R kernel, though there is an emerging community of aficianados for different languages and JupyterLab. In order to use this kernel, you first have to have a version of R installed. 

## Installing R generally

You might get lucky with the Environment settings within [Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/). This guide from Anaconda shows how to do this visually. It worked for me on Mac on the first try. It is also an alternative way to manage environments generally. I notice that my geo-env environment shows up here as well. 

Once you create a new environment through this approach, you can use it from Anaconda Navigator OR by activating that environment from the command prommpt. But be forewarned, it's an entirely different Anaconda, which means that it will need to reinstall things like Jupyter Lab, Seaborn, etc...

The instructions below are an alternative to build an R kernel to use with your 'base' Jupyter install. 

## Installing R on Windows 
First go to [CRAN](https://cloud.r-project.org/), the Comprehensive R Archive Network, and download the latest version of R, which will be 4.+ and 64-bit. Install this file on your computer. 

Then you should probably check that it is working. Much like how you can use Python from the command prompt in PowerShell, you can use R. But the R installation won't place the executable file on your path by default, so navigate to it, which will be in a directory similar to the following: 
~~~ bash
> cd "C:\Program Files\R\R-4.0.3\bin"
~~~

Then run the command in that folder, which is `R.exe`. However, within PowerShell, you'll need to append `.\` so this is the command in the terminal: 

~~~ bash
> .\R.exe
~~~
Then the prompt should change slightly and you'll get a preamble that let's you know you're in R. So far so good.  But now let's get out of here! Type `exit()` to quit. Now at the same prompt you should be able to type: (remember you should be in the `base` environment where you type `jupyter lab`) 

~~~ bash 
> conda install -c r r-irkernel
~~~ 
Then for me it worked immediately upon restarting Jupyter lab in a PowerShell / Terminal. Others have had slightly more complex experiences, but I think those are remnants of earlier versions of Jupyter Lab and R. 

To note, the earlier experiences suggest instead installing the dependencies from within R. This would involve running the PowerShell as administrator and then entering the following commands. Please note, this might involve further installs such as [RTools](https://cran.r-project.org/bin/windows/Rtools/) and/or [Git](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git) separately. I would recommend trying through conda first. 

~~~ R
install.packages("devtools")
devtools::install_github("IRkernel/IRkernel")
IRkernel::installspec()
~~~

After either installing through `conda` or through `R`, then simply restart Jupyter and you should see a new type of Kernel when using the Jupyter Launcher.


# Working with the OxCOVID19 database examples online

To make use of our new packages we can explore some of the example code from the OxCOVID19 database GitHub. There they have a number of useful examples that employ `geopandas`, `geoplot`, and some related packages. For example, I had a look at the following example of [COVID-19 cases in Germany](https://github.com/covid19db/examples/blob/master/data-plots/confirmed-adm-1-level-germany.ipynb). 

You will notice that they did not place all the import statements at the top (tsk tsk). so I had to look through them in order to check what we need to run for our new environment. We will need to install the following:
- `geopands` (which should be done either through the fresh install approach, should you dare; or the virtual environment) 
- `geoplot`
- `psycopg2` 
- `termcolor` 

For `psycopg2` I had to go not to `conda` but to `pip` which would then build it from source for our version. 

Below are a series of statements that you can run within a Jupyter notebook, however, you should ensure that this notebook is the proper environment that has `geopandas` installed. So that might either involve the fresh install, or by opening **this** notebook using the `geo-env` kernel. 

If you choose instead to do this through the command line (which will give more prompt feedback) , be sure to first activate your virtual environment so that your terminal will look like: 

~~~ bash
(geo-env) $ Work@Personals-MacBook-Air . conda% conda install <package>
~~~

and **not**: 

~~~ bash
(base) $ Work@Personals-MacBook-Air . conda% conda install <package>
~~~

If you have created a virtual kernel to install `geopandas`. Then here you would want to run: 

~~~ bash
conda install geoplot -c conda-forge
pip install psycopg2
pip install termcolor
~~~ 



In [1]:
try:
    import geoplot
except ModuleNotFoundError: 
    !conda install --yes --prefix {sys.prefix} geoplot -c conda-forge
    import geoplot

In [ ]:
try:
    import psycopg2
except ModuleNotFoundError: 
    !{sys.executable} -m pip install psycopg2
    import psycopg2

In [ ]:
try:
    import termcolor
except ModuleNotFoundError: 
    !{sys.executable} -m pip install termcolor
    import termcolor

# Calling a lifeline 

This is really a challenging and time consuming part of research. For many new packages from tensorflow and pytorch to geopandas, bokeh and qgis, there are a lot of moving pieces from different players all constantly updating their software in tandem. 

I've given lots of approaches. The `Anaconda Navigator` environments approach is the most abstract (see link in the Installing R section) for an example. It does tend to work across platforms. However, lots of times dependencies simply can't be resolved or other cascading installs happen. For that reason, I have shown 
- how to create the simplified venv virtual environment, 
- how to use `pip` (including how to export a list of installed packages)
- how to create a virtual environment with conda (and the reminder about challenges to `activate` on windows). 
- ways to install the R kernel / environment for Jupyter Lab 
- requirements for opening the OxCOVID geopandas data. 

These are all really a challenge and I hope this gives you some additional help and focus on this work. I will say that once I got `geopandas` working and was able to play with some maps, getting variations on these has been really swift. This is not really spatial modeling, per se, but merely spatial representation. But it is an excellent first start on any GIS journey.

If you have additional approaches or challenges, post them in the issue tracker for this document. I'll try my best to update this. For example, I'm aware that there are also ways to install packages through [`docker`](https://github.com/jupyter/docker-stacks) which is a little more abstract and high level, but it is likely to become increasingly relevant and scalable. 

# Useful links 

In preparing this document, I drew upon tons of really helpful but usually very terse guides for a variety of approaches. Please note that these are all somewhat partial and can even go out of date while still showing up in search results. 

General package websites
- https://conda-forge.org/
- https://pypi.org/project/psycopg2/
- https://git-scm.com/book/en/v2/Getting-Started-Installing-Git
- https://docs.conda.io/en/latest/miniconda.html
- https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html

Virtual Environments 
- https://janakiev.com/blog/jupyter-virtual-envs/
- https://www.freecodecamp.org/news/manage-multiple-python-versions-and-virtual-environments-venv-pyenv-pyvenv-a29fb00c296f/
- https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove
- https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

Linking Jupyter to Kernels
- https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084
- https://github.com/ContinuumIO/anaconda-issues/issues/11148

Installing and Linking R
- https://realpython.com/python-wheels/#python-packaging-made-better-an-intro-to-python-wheels
- https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/
- https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/

Geopandas installations
- https://gis.stackexchange.com/questions/358889/geopandas-installation-errors
- https://residentmario.github.io/geoplot/installation.html
